In [1]:
import os
import tf_agents.policies
import tf_agents.specs
from tf_agents.environments import suite_gym, parallel_py_environment
from tf_agents.environments import tf_py_environment
from tf_agents.metrics import tf_metrics
from tf_agents.policies import policy_saver
from tf_agents.policies.tf_py_policy import TFPyPolicy
from tf_agents.replay_buffers import tf_uniform_replay_buffer, episodic_replay_buffer
from tf_agents.drivers import dynamic_episode_driver, dynamic_step_driver
import tensorflow as tf
from tf_agents.trajectories.policy_step import PolicyStep
from policies import SavedTFPolicy
tf.config.set_visible_devices([], 'GPU')  #  allows testing during training
from tf_agents.trajectories import time_step as ts, policy_step, trajectory
from reinforcement_learning import labeling_functions
labeling_function = labeling_functions['LunarLanderContinuous-v2']
from util.io.dataset_generator import map_rl_trajectory_to_vae_input
from util.io.dataset_generator import ErgodicMDPTransitionGenerator
import tensorflow_probability as tfp
tfd = tfp.distributions

In [2]:
py_env = suite_gym.load('LunarLanderContinuous-v2')
py_env.reset()
tf_env = tf_py_environment.TFPyEnvironment(py_env)
tf_env.time_step_spec()

TimeStep(step_type=TensorSpec(shape=(), dtype=tf.int32, name='step_type'), reward=TensorSpec(shape=(), dtype=tf.float32, name='reward'), discount=BoundedTensorSpec(shape=(), dtype=tf.float32, name='discount', minimum=array(0., dtype=float32), maximum=array(1., dtype=float32)), observation=BoundedTensorSpec(shape=(8,), dtype=tf.float32, name='observation', minimum=array(-3.4028235e+38, dtype=float32), maximum=array(3.4028235e+38, dtype=float32)))

In [5]:
def display_labeling(trajectory):
    label = labeling_functions['LunarLanderContinuous-v2'](trajectory.observation)
    if tf.reduce_any(label[..., 2]) and tf.reduce_any(label[..., 6]):
        print('close to the lunar pad with high speed')
    if not tf.reduce_any(label[..., 7]):
        print('unsafe lander angle')
    if tf.reduce_any(label[..., 2]) and not tf.reduce_any(label[..., 8]):
        print('close to the lunar pad with unsafe lander angle')
    if tf.reduce_any(label[..., 1]):
        print('lander too close to the edge of the frame')

reward_metric = tf_metrics.AverageReturnMetric()
sac_policy_dir = '../saves/LunarLanderContinuous-v2/policy'
saved_policy = SavedTFPolicy(sac_policy_dir)
dynamic_episode_driver.DynamicEpisodeDriver(tf_env, saved_policy, num_episodes=5,
                                            observers=[
                                                display_labeling,
                                                lambda _: py_env.render(mode='human'),
                                                reward_metric
                                            ]).run()
reward_metric.result()

<tf.Tensor: shape=(), dtype=float32, numpy=263.88812>

In [9]:
import variational_action_discretizer

vae_mdp = variational_action_discretizer.load(
    # "../../saves/Pendulum-v0/models/vae_LS12_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.70-0.50_1e-06-2e-06/policy/action_discretizer/LA3_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.50-0.33_1e-06-2e-06_params=full_vae_optimization/step140000/eval_elbo-3.784"
    # "../../saves/Pendulum-v0/models/vae_LS13_MC1_ER20.0-decay=2e-05-min=-10_KLA0.0-growth=1e-06_TD0.70-0.50_1e-06-2e-06/policy/action_discretizer/LA5_MC1_ER20.0-decay=2e-05-min=-10_KLA0.0-growth=1e-06_TD0.25-0.17_1e-06-2e-06_params=full_vae_optimization/step270000/eval_elbo-1.704"
    # "../../saves/Pendulum-v0/models/vae_LS13_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.70-0.50_1e-06-2e-06/permissive_variance_policy-multiplier=10.0/action_discretizer/LA5_MC1_ER20.0-decay=7.5e-05-min=-10_KLA0.0-growth=1e-06_TD0.25-0.17_1e-06-2e-06_params=full_vae_optimization/step70000/eval_elbo-7.265"
    # "../../saves/Pendulum-v0/models/vae_LS12_MC3_CER100.0-decay=1e-05_KLA1e-06-growth=1e-07_TD0.95-0.90_1e-06-2e-06/policy/action_discretizer/LA3_MC3_CER100.0-decay=1e-05_KLA1e-06-growth=1e-07_TD0.50-0.33_1e-06-2e-06_params=full_vae_optimization/step1010000/eval_elbo11.174"
    '../../saves/LunarLanderContinuous-v2/models/vae_LS20_MC1_ER10.0-decay=1e-05-min=0_KLA0.0-growth=5e-05_TD0.67-0.50_1e-06-2e-06_seed=20421/policy/action_discretizer/LA4_MC1_ER10.0-decay=1e-05-min=0_KLA0.0-growth=5e-05_TD0.33-0.22_1e-06-2e-06_params=full_vae_optimization-relaxed_state_encoding/base',
    step=880000
)
print("VAE MDP loaded")

vae_mdp.eval_policy(eval_env=py_env, labeling_function=labeling_function, num_eval_episodes=20, render=True)

VAE MDP loaded
eval policy -92.45823


<tf.Tensor: shape=(), dtype=float32, numpy=-92.45823>